### Reflexion
Do the research on a given topic. Collect data from LLM -> Give self feedback -> Get online search query from data -> Collect data from online search -> get result of combined data -> do the evaluation -> again get data from tool (online) and repeat the process and finallly give output.

In [ ]:
import json
import random
from collections import defaultdict
import datetime

In [ ]:
from typing import List
from langchain_core.pydantic_v1 import BaseModel, Field


## Schema
* Reflection class is a critique and having reflection schema
* AnswerQuestion class has schema of answer, which includes reflection as one of the field
* ReviseAnser is the revisor, derived from AnswerQuestion. Having reference as field which will consist citation.

In [ ]:
class Reflection(BaseModel):
    """reflection on the initial answer"""
    missing: str = Field(description="Critique of what is missing.")
    superfluous: str = Field(description="Critique of what is superfluous")


class AnswerQuestion(BaseModel):
    """Answer the question."""

    answer: str = Field(description="~250 word detailed answer to the question.")
    missing: str = Field(description="Critique of what is missing.")
    superfluous: str = Field(description="Critique of what is superfluous")
    search_queries: List[str] = Field(
        description="1-3 search queries for researching improvements to address the critique of your current answer."
    )


# Forcing citation in the model encourages grounded responses
class ReviseAnswer(AnswerQuestion):
    """Revise your original answer to your question."""

    references: List[str] = Field(
        description="Citations motivating your updated answer."
    )


In [ ]:
from langchain_core.output_parsers.openai_tools import (
    PydanticToolsParser,
    JsonOutputToolsParser,
)
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_google_genai import ChatGoogleGenerativeAI

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
google_api_key = os.environ["GOOGLE_API_KEY"]
tavily_api_key = os.environ["TAVILY_API_KEY"]

## Gemini-1.5-pro LLM

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro")

## First Node

In [ ]:
actor_prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are expert researcher.
                Current time: {time}
                1. {first_instruction}
                2. Reflect and critique your answer. Be severe to maximize improvement.
                3. Recommend search queries to research information and improve your answer."""
        ),
        MessagesPlaceholder(variable_name="messages"),
        #("system", "Answer the user's question above using the required format."),
    ]
).partial(
    time=lambda: datetime.datetime.now().isoformat(),
)

In [ ]:

first_responder = actor_prompt_template.partial(
    first_instruction="Provide a detailed ~250 word answer."
) | llm.bind_tools(tools=[AnswerQuestion], tool_choice="AnswerQuestion")
validator = PydanticToolsParser(tools=[AnswerQuestion])

## Second Node

In [ ]:
revise_instructions = """Revise your previous answer using the new information.
    - You should use the previous critique to add important information to your answer.
        - You MUST include numerical citations in your revised answer to ensure it can be verified.
        - Add a "References" section to the bottom of your answer (which does not count towards the word limit). In form of:
            - [1] https://example.com
            - [2] https://example.com
    - You should use the previous critique to remove superfluous information from your answer and make SURE it is not more than 250 words.
"""


In [ ]:
# Revisor node
revisor = actor_prompt_template.partial(
    first_instruction=revise_instructions
) | llm.bind_tools(tools=[ReviseAnswer], tool_choice="ReviseAnswer")


## Online search with Tavily

In [ ]:
## Tavily Search

from langchain_community.tools.tavily_search import TavilySearchResults
tavily_tool = [TavilySearchResults(tavily_api_key = tavily_api_key, max_results=2)]

In [ ]:
from langgraph.prebuilt import ToolInvocation, ToolExecutor
tool_executor = ToolExecutor(tavily_tool)

In [ ]:
from chains import parser
from langchain_core.messages import AIMessage, BaseMessage, ToolMessage, HumanMessage, SystemMessage

## Node for online search 
def execute_tools(state: List[BaseMessage]) -> List[BaseMessage]:
    # Take last tool from state
    tool_invocation: AIMessage = state[-1]

    # Invoke tool
    parsed_tool_calls = parser.invoke(tool_invocation)

    # Parse the output and get search queries and ids
    ids = []
    tool_invocations = []
    for parsed_call in parsed_tool_calls:
        for query in parsed_call["args"]["search_queries"]:
            tool_invocations.append(
                ToolInvocation(
                    tool="tavily_search_results_json",
                    tool_input=query,
                )
            )
            ids.append(parsed_call["id"])


    # execute the search tool.
    outputs = tool_executor.batch(tool_invocations)

    # now map the output with id (as a key) with query and output
    outputs_map = defaultdict(dict)
    for id_, invocation, output in zip(ids, tool_invocations, outputs):
        outputs_map[id_][invocation.tool_input] = output

    tool_messages = []
    for id_, query_outputs in outputs_map.items():
        tool_messages.append(
            ToolMessage(content=json.dumps(query_outputs), tool_call_id=id_)
        )

    return tool_messages


## Create the Graph

In [ ]:
from langgraph.graph import END, MessageGraph

MAX_ITERATIONS = 2
builder = MessageGraph()

## Add nodes
builder.add_node("draft", first_responder)
builder.add_node("execute_tools", execute_tools)
builder.add_node("revise", revisor)

## Add edges. 
## Draft output will have research as well as crtique/feedback. Then it will go to online search.
## Online search result wil go to revise. Based on revise prompt llm will revise the content and reference.
builder.add_edge("draft", "execute_tools")
builder.add_edge("execute_tools", "revise")

In [ ]:
## Add conditional edge.
## Revisor will send for revision to execution tool by MAX_ITERATIONS times.

def event_loop(state: List[BaseMessage]) -> str:
    # AIMessage will keep adding in state on every execution of node.
    # Before first call of this method, there are two AIMessage already there by earlier node execution.
    count_tool_visits = sum(isinstance(item, AIMessage) for item in state)
    #print("count_tool_visits = ", count_tool_visits)
    if count_tool_visits -2 > MAX_ITERATIONS:
        return END
    return "execute_tools"




In [ ]:
builder.add_conditional_edges("revise", event_loop)

In [ ]:
## Set entry point and build the graph
builder.set_entry_point("draft")
graph = builder.compile()


In [ ]:
## Execute the graph

res = graph.invoke(
    ["Write about Agentic AI and real life use cases, list startups that do that and raised capital."]
)
print(res[-1].tool_calls[0]["args"]["answer"])

In [ ]:
print("References = ", res[-1].tool_calls[0]["args"]["references"])